In [1]:
import pandas as pd 
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Dataset, Reader
import time
from surprise.prediction_algorithms import SVD
from Functions import SVD_Predict


path = os.path.join(os.getcwd(), 'anime_data')
anime_rec = pd.read_csv(os.path.join(path, 'anime_rec.csv'))
rating = pd.read_csv(os.path.join(path, 'rating.csv'))
anime_rec_2 = anime_rec[['anime_id','name']]
rating_withnames = pd.merge(rating, anime_rec_2, on = 'anime_id')
all_data = rating_withnames[rating_withnames['rating'] != -1]
#shuffle dataset adjust frac=0.1 for 10% of dataset,0.5 for 50%, 1 for 100% etc.
final_data = all_data.sample(frac=1).reset_index(drop=True) 
final_data = final_data.drop('name', axis=1)

reader = Reader(rating_scale=(1, 10))
from surprise.model_selection import train_test_split
data = Dataset.load_from_df(final_data, reader)
#train, test = train_test_split(data)
train = data.build_full_trainset()


#Where the actual code starts




In [15]:
from surprise.dump import dump, load
algo = SVD()
algo.fit(train)


In [16]:
dump(os.path.join(os.getcwd(), '100%_data.txt'), algo=algo)

In [17]:
d = load(os.path.join(os.getcwd(), '100%_data.txt'))
algo = d[-1]
predictions = SVD_Predict(algo, [(1, 3), (20, 1)])
df = pd.DataFrame(predictions, columns=['anime_id', 'rating'])
new_data = pd.merge(df.nlargest(3, 'rating'), anime_rec_2, left_on=['anime_id'], right_on=['anime_id'])
new_data = new_data[['name', 'anime_id', 'rating']]


,name,anime_id,rating
0,Persona 3 the Movie 1: Spring of Birth,14407,9.320490
1,Code Geass: Hangyaku no Lelouch,1575,9.074894
2,Code Geass: Hangyaku no Lelouch R2,2904,9.017928
